### package installation

In [6]:
! pip install langchain


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
! pip install langchain-openai


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
! pip install tiktoken


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Setting Environment

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
# print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

In [5]:
from langchain_openai import ChatOpenAI

TEMPERATURE = 0.1
MODEL_NAME = "gpt-3.5-turbo"
# gpt-4o-mini 가 제일 좋은건데 과금이슈로 일단은 3.5-turbo를 사용 ㅎㅎ

llm = ChatOpenAI(
    temperature = TEMPERATURE,
    model_name = MODEL_NAME
)

# logProb는 token (문장을 구성하는 단어, 문자) 확률의 로그값을 return
# => token probabilty? : 모델이 그 토큰을 예측할 확률?

In [28]:
from langchain_core.prompts import ChatPromptTemplate

# prompt template를 구성하여 template에 질문 요소를 삽입할 수 있음
prompt = ChatPromptTemplate.from_template("내가 하는 질문을 3문장 이상으로 구체적으로 답변해줘 <question> : {input}")

In [29]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()

In [30]:
# chain.stream => 한 토큰씩 불러오기

for token in chain.stream({"input" : "2024 kbo 한국시리즈"}):
    print(token, end="", flush=True)

2024 KBO 한국시리즈는 한국 프로야구 리그인 KBO리그에서 열리는 시즌 중 가장 중요한 이벤트로, 리그 우승팀과 준우승팀이 직접 대결하는 시리즈 경기를 말합니다. 이 대결은 최대 7경기까지 이어질 수 있으며, 최종 승리팀이 한국시리즈 챔피언으로 선정됩니다. 2024년에는 어떤 팀이 우승을 차지할지 예측하기는 어렵지만, 역사적으로 강세를 보인 팀들이 유력한 후보로 거론될 것입니다.

In [31]:
# chain.invoke => 통으로 응답 하나 불러오기 (사용자에게 응답 시간이 길다고 느껴질 수 있음)

chain.invoke({"input" : "홍창기"})

'홍창기라는 사람에 대해 궁금하신가요? 홍창기는 한국의 작가이며, 그의 작품은 주로 사회 문제나 역사적 사건을 다루고 있습니다. 그의 작품은 독특한 시선과 표현으로 많은 독자들의 사랑을 받고 있습니다. 또한, 홍창기는 문학상을 수상한 경력도 가지고 있어 한국 문학계에서 주목받는 작가 중 한 명입니다.'

In [ ]:
# chain.batch => input dictionary를 리스트화하여 한번에 여러개를 질문할 수 있도록 함
# max_concurrency 파라미터를 통해 동시에 처리할 수 있는 최대 작업 수를 설정 (병렬적으로 처리하는건가?)

# chain.batch([{"input" : "오지환"}, {"input" : "손주영"}])

In [32]:
# runnableParalell => 병렬 요청을 처리

from langchain_core.runnables import RunnableParallel

# (1) 선수의 소속 팀을 소개하는 체인
chain1 = (ChatPromptTemplate.from_template("{player}의 소속팀은 어디야?")
          | llm
          | StrOutputParser())

# (2) 선수의 포지션을 소개하는 체인
chain2 = (ChatPromptTemplate.from_template("{player}의 포지션은 뭐야?")
          | llm
          | StrOutputParser())

combined = RunnableParallel(team=chain1, position=chain2)

In [33]:
combined.invoke({"player" : "홍창기"})

{'team': '홍창기는 현재 대한민국 프로축구팀인 성남FC에 소속되어 있습니다.',
 'position': '홍창기는 축구 선수이며, 포지션은 공격수입니다.'}

In [8]:
# runnableLambda => 사용자 정의 함수를 매핑

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime

def get_today(a):
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")

In [14]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 야구선수 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)


llm = ChatOpenAI(temperature=TEMPERATURE, model_name="gpt-4o")

# chain 을 생성합니다.
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
print(chain.invoke(3))

10월 28일이 생일인 야구선수 세 명을 소개하겠습니다.

1. **호세 알투베 (Jose Altuve)**  
   - 생년월일: 1990년 10월 28일

2. **노아 신더가드 (Noah Syndergaard)**  
   - 생년월일: 1992년 10월 28일

3. **앤드류 베일리 (Andrew Bailey)**  
   - 생년월일: 1984년 10월 28일

이 선수들은 메이저 리그 베이스볼(MLB)에서 활약한 선수들입니다.
